## 2.2 数据预处理

### 2.2.1. 读取数据集

- 我们首先创建一个人工数据集house_tiny.csv

In [2]:
import os

os.makedirs(os.path.join('..', 'Data'), exist_ok=True)
data_file = os.path.join('..', 'Data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

- read_csv 读取csv文件

In [4]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


### 2.2.2. 处理缺失值

- 通过位置索引iloc，我们将 data 分成 inputs 和 outputs，其中前者为 data的前两列，而后者为 data的最后一列
- 对于 inputs 中缺少的的数值，我们调用fillna函数，用同一列的均值替换 “NaN” 项

In [7]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
inputs, outputs

(   NumRooms Alley
 0       3.0  Pave
 1       2.0   NaN
 2       4.0   NaN
 3       3.0   NaN,
 0    127500
 1    106000
 2    178100
 3    140000
 Name: Price, dtype: int64)

- get_dummies 是利用pandas实现one hot encode的方式
- 由于 “Alley”列只接受两种类型的类别值 “Alley” 和 “NaN”，pandas 可以自动将此列转换为两列 “Alley_Pave” 和 “Alley_nan”。

In [11]:
inputs = pd.get_dummies(inputs, dummy_na=True)
inputs

,NumRooms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


### 2.2.3. 转换为张量格式

**inputs 和 outputs 中的所有条目都是数值类型，它们可以转换为张量格式**

In [14]:
import torch

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

**Pandas去除nan值占百分比大于80的列**

In [27]:
import numpy as np
import pandas as pd
df = pd.DataFrame({"Col_1":[1]*9+[np.nan]*2, "Col_2":[2]*11, "Col_3":[np.nan]*11, "Col_4":[4]*11, "Col_5":[np.nan]*10+[5]}) # 一个包含两列nan值占比大于80%的
print(df)
def drop_col(df, cutoff=0.2): # 如果这一列中有80%以上的缺失值，那么就从df中删除这一列
    n = len(df)
    cnt = df.count() # 对列进行非nan值计数
    cnt = cnt / n # 求出非nan值的百分比
    # maxNan=  cnt if cnt > maxNan else maxNan
    return df.loc[:, cnt[cnt >= cutoff].index] # 根据cnt记录的百分比，过滤出cnt百分百大于等于0.2的（也就是去掉nan值大于0.8的索引），然后对df进行选择，行所有，列为满足要求的cnt的索引。
df = drop_col(df)
print(df)

    Col_1  Col_2  Col_3  Col_4  Col_5
0     1.0      2    NaN      4    NaN
1     1.0      2    NaN      4    NaN
2     1.0      2    NaN      4    NaN
3     1.0      2    NaN      4    NaN
4     1.0      2    NaN      4    NaN
5     1.0      2    NaN      4    NaN
6     1.0      2    NaN      4    NaN
7     1.0      2    NaN      4    NaN
8     1.0      2    NaN      4    NaN
9     NaN      2    NaN      4    NaN
10    NaN      2    NaN      4    5.0
    Col_1  Col_2  Col_4
0     1.0      2      4
1     1.0      2      4
2     1.0      2      4
3     1.0      2      4
4     1.0      2      4
5     1.0      2      4
6     1.0      2      4
7     1.0      2      4
8     1.0      2      4
9     NaN      2      4
10    NaN      2      4


**删除缺失值最多的列**

In [36]:
import numpy as np
import pandas as pd
df = pd.DataFrame({"Col_1":[1]*9+[np.nan]*2, "Col_2":[2]*11, "Col_3":[np.nan]*11, "Col_4":[4]*11, "Col_5":[np.nan]*10+[5]}) # 一个包含两列nan值占比大于90%的
print(df)
maxNan = 0.0
for index, row in df.iteritems():
    maxNan = df[index].count() if maxNan <= df[index].count() else maxNan
cnt = df.count()
n = len(df)
df = df.loc[:, cnt[cnt > n - maxNan].index]
cnt[cnt > n - maxNan].index, cnt, df

    Col_1  Col_2  Col_3  Col_4  Col_5
0     1.0      2    NaN      4    NaN
1     1.0      2    NaN      4    NaN
2     1.0      2    NaN      4    NaN
3     1.0      2    NaN      4    NaN
4     1.0      2    NaN      4    NaN
5     1.0      2    NaN      4    NaN
6     1.0      2    NaN      4    NaN
7     1.0      2    NaN      4    NaN
8     1.0      2    NaN      4    NaN
9     NaN      2    NaN      4    NaN
10    NaN      2    NaN      4    5.0


(Index(['Col_1', 'Col_2', 'Col_4', 'Col_5'], dtype='object'),
 Col_1     9
 Col_2    11
 Col_3     0
 Col_4    11
 Col_5     1
 dtype: int64,
     Col_1  Col_2  Col_4  Col_5
 0     1.0      2      4    NaN
 1     1.0      2      4    NaN
 2     1.0      2      4    NaN
 3     1.0      2      4    NaN
 4     1.0      2      4    NaN
 5     1.0      2      4    NaN
 6     1.0      2      4    NaN
 7     1.0      2      4    NaN
 8     1.0      2      4    NaN
 9     NaN      2      4    NaN
 10    NaN      2      4    5.0)